# GRIT-Atlas: Single-Cell Resource Construction and Annotation Pipeline

This notebook delineates the computational workflow for constructing the Glioblastoma Resistance Insights from Treatment Atlas (GRIT-Atlas). The pipeline encompasses standardized quality control (QC), multi-batch integration via BBKNN, dimensionality reduction, and primary lineage annotation for nearly one million cells.

## 1. Environment Setup and Dataset Definition

We initialize the analysis environment and define the 17 discovery and validation cohorts integrated into the final atlas.

In [ ]:
import os
import pandas as pd
import scanpy as sc
from multiprocessing import Pool
import time
import infercnvpy as cnv
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sparse
import scipy.io as sio

# Cohorts included in the GRIT-Atlas (n=299 samples)
filenames = [
    "NatCancer_Wang_2022", "NatCommun_Abdelfattah_2022", "Science_Christina_2025",
    "NatCancer_Mei_2023", "NatCancer_Richards_2021", "GusuCohort",
    "NatCommun_Lee_2021", "NatNeurosci_Antunes_2021", "JCIInsight_Xie_2021",
    "NatCommun_Ravi_2022", "CancerCell_LeBlanc_2022", "CancerDiscov_Wang_2019",
    "GenomeMed_Chen_2021", "AdvSci_Xiao_2019", "NatCommun_AlDalahmah_2023",
    "GenomeMed_Yuan_2018", "FrontImmunol_Xiao_2022"
]

## 2. Standardized Quality Control (QC) & Preprocessing

To ensure data high-fidelity, we implemented a strict QC pipeline to filter out low-quality cells, doublets, and technical artifacts.

In [ ]:
def InitialFilter(filename):
    start_time = time.time()
    file_path = os.path.join("GRIT_Atlas", filename, 'Initialadata.h5ad')
    adata = sc.read(file_path)
    
    # 1. Feature annotation for QC
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
    adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True)
    
    # 2. Strict filtering based on manuscript criteria
    sc.pp.filter_cells(adata, min_genes=500)
    sc.pp.filter_genes(adata, min_cells=5)
    adata = adata[adata.obs.nFeature_RNA < 5000, :]
    adata = adata[adata.obs.pct_counts_mt < 10, :]
    adata = adata[adata.obs.pct_counts_ribo < 50, :]
    adata = adata[adata.obs.pct_counts_hb < 0.05, :]

    # 3. Doublet identification using Scrublet
    sc.pp.scrublet(adata, batch_key="orig.ident", threshold=0.3, n_prin_comps=20,
                   normalize_variance=False, log_transform=False, expected_doublet_rate=0.05)
    adata = adata[adata.obs.predicted_doublet == False, :]
    
    # 4. Normalization and log-transformation (target_sum=1e4)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    # 5. Technical effect regression (UMI counts and sample identity)
    sc.pp.regress_out(adata, ["orig.ident", "nCount_RNA", "nFeature_RNA"])

    results_file = os.path.join("GRIT_Atlas", filename, 'InitialFilter.h5ad')
    adata.write(results_file)

    total_time = time.time() - start_time
    print(f"Dataset {filename} processed in {total_time:.2f}s")
    return adata

## 3. Large-scale Dataset Integration

Individual datasets were merged into a unified object. Samples with fewer than 500 retained cells were excluded to ensure statistical robustness.

In [ ]:
# Load all preprocessed datasets
adata_list = [sc.read(os.path.join("GRIT_Atlas", file, 'InitialFilter.h5ad')) for file in filenames]

# Stepwise concatenation
adata_concat = adata_list[0]
for adata_next in adata_list[1:]:
    adata_concat = adata_concat.concatenate(adata_next, join='inner')

# Exclude low-quality samples (n < 500 cells)
value_counts_df = adata_concat.obs['orig.ident'].value_counts().reset_index()
value_counts_df.columns = ['orig.ident', 'count']
sample_names = value_counts_df[value_counts_df['count'] >= 500]['orig.ident'].tolist()
adata_concat = adata_concat[adata_concat.obs['orig.ident'].isin(sample_names)]

adata_concat.write("GRIT_Atlas/adata_concat.h5ad")

## 4. Batch Correction & Dimensionality Reduction

We employed the BBKNN (Batch Balanced K-Nearest Neighbors) algorithm to mitigate batch effects while preserving biological variation.

In [ ]:
# 1. Feature scaling and selection (Top 2000 HVGs)
sc.pp.scale(adata_concat, max_value=10)
sc.pp.highly_variable_genes(adata_concat, min_mean=0.0125, max_mean=3, min_disp=0.5, n_top_genes=2000)
adata_concat = adata_concat[:, adata_concat.var.highly_variable]

# 2. PCA and BBKNN Integration (30 PCs)
sc.pp.pca(adata_concat, n_comps=30)
sc.external.pp.bbknn(adata_concat, batch_key="batch", n_pcs = 30)

# 3. UMAP Embedding and Multi-resolution Leiden Clustering
sc.tl.umap(adata_concat)
resolutions = np.arange(0.1, 2.1, 0.1)
for res in resolutions:
    key = f"leiden_res{str(round(res, 1)).replace('.', '_')}"
    sc.tl.leiden(adata_concat, key_added=key, resolution=res, random_state=0, flavor="igraph")

adata_concat.write("GRIT_Atlas/adata_concat_integrate.h5ad")

## 5. Primary Lineage Annotation

Cells were categorized into 7 major lineages based on a predefined dictionary of canonical markers at a resolution of 2.0.

In [ ]:
# Mapping Leiden clusters to major lineages
leiden_to_celltype = {
    '0': "Lymphocyte", '1': "Lymphocyte", '2': "Lymphocyte", 
    '3': "Myeloid", '4': "Malignant", '5': "Myeloid", 
    '6': "Myeloid", '7': "Myeloid", '8': "Myeloid", 
    '9': "Malignant", '10': "Astrocyte/Neuron", '11': "Malignant", 
    '12': "Myeloid", '13': "Myeloid", '14': "Myeloid", 
    '15': "Oligodendrocyte", '16': "Malignant", '17': "Malignant", 
    '18': "Malignant", '19': "Astrocyte/Neuron", '20': "Astrocyte/Neuron", 
    '21': "Myeloid", '22': "Myeloid", '23': "Myeloid", 
    '24': "Malignant", '25': "Malignant", '26': "Malignant", 
    '27': "Stroma", '28': "Astrocyte/Neuron", '29': "Astrocyte/Neuron", 
    '30': "Malignant", '31': "Endothelial", '32': "Myeloid", 
    '33': "Malignant", '34': "Oligodendrocyte"
}

adata_concat.obs['celltype'] = adata_concat.obs['leiden_res2_0'].replace(leiden_to_celltype)
adata_concat.write("GRIT_Atlas/adata_concat_annotated_fullgenes.h5ad")

## 6. Sub-lineage Extraction for Deep Phenotyping

Major cell compartments were isolated and exported for downstream high-resolution subclustering and functional analysis (e.g., cNMF for malignant cells).

In [ ]:
def save_lineage_data(adata, cluster_name, base_path="GRIT_Atlas/FirstRoundAnnotation/"):
    folder_path = os.path.join(base_path, cluster_name)
    os.makedirs(folder_path, exist_ok=True)

    # Export Metadata and Gene information
    adata.obs.to_csv(os.path.join(folder_path, "cellinfo.csv"))
    adata.var.to_csv(os.path.join(folder_path, "geneinfo.csv"))

    # Export Sparse Matrix for R-platform (Seurat/RCTD) compatibility
    mtx = adata.layers['counts'].T if 'counts' in adata.layers else adata.X.T
    sparse_matrix = sparse.csr_matrix(mtx)
    sio.mmwrite(os.path.join(folder_path, "sparse_matrix.mtx"), sparse_matrix)

# Subsetting lineages
lineages = ["Malignant", "Myeloid", "Lymphocyte", "Stroma", "Endothelial"]
for name in lineages:
    subset = adata_concat[adata_concat.obs['celltype'] == name]
    save_lineage_data(subset, name)